# Introduction to Community Modeling

Author: Daniel Machado, NTNU

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).

- You will use the [ReFramed](https://github.com/cdanielmachado/reframed) python library for metabolic modeling. 

In [1]:
from reframed.solvers import solvers

solvers

{'gurobi': reframed.solvers.gurobi_solver.GurobiSolver,
 'optlang': reframed.solvers.optlang_solver.OptLangSolver}

## Step 1: Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

![synthetic community](../files/synthetic_community.png)

As usual, we start by loading the model for the wild-type:

In [44]:
from reframed import load_cbmodel
wildtype = load_cbmodel('../models/non-ec/e_coli_core.xml.gz')

In [45]:
bt = load_cbmodel('../models/non-ec/Bacteroides_thetaiotaomicron_VPI_5482.xml')

In [26]:
bu = load_cbmodel('../models/non-ec/Bacteroides_uniformis_ATCC_8492.xml')

In [31]:
ec = load_cbmodel('../models/non-ec/Escherichia_coli_ED1a.xml')

Now we create our two mutants (`glc_ko` and `nh4_ko`):

In [32]:
glc_ko = wildtype.copy()
glc_ko.id = 'glc_ko'
glc_ko.set_flux_bounds('R_GLCpts', 0, 0)

nh4_ko = wildtype.copy()
nh4_ko.id = 'nh4_ko'
nh4_ko.set_flux_bounds('R_NH4t', 0, 0)

**ReFramed** has some basic functionality for working with microbial communities, one is the `Community` class to create microbial communities from a list of models of individual species: 

In [40]:
from reframed import Community
community = Community('ecoli_pair', [bt,bu,ec])

This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [41]:
from reframed import Environment

M9 = Environment.from_model(wildtype)

print(f"Environment compounds: {', '.join(M9.get_compounds())}")

Environment compounds: co2, glc__D, h, h2o, nh4, o2, pi


## Step 2: Simulation using (conventional) FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [42]:
from reframed import FBA

super_oganism = community.merged_model
solution = FBA(super_oganism, constraints=M9)

print(solution)
solution.show_values(pattern='R_EX')

Objective: -0.0
Status: Optimal




/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_EX_ac_e' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_EX_acald_e' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_EX_akg_e' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/python3.9/site-packages/reframed/solvers/gurobi_solver.py:236: UserWarning: Constrained variable 'R_EX_co2_e' not previously declared
  warn(f"Constrained variable '{r_id}' not previously declared")
/home/mmartins/anaconda3/envs/tmcom/lib/pyt

We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the (non-zero) fluxes for each organism:

In [20]:
solution.show_values(pattern='_glc_ko', sort=True)

R_ACALDt_glc_ko -102.26
R_H2Ot_glc_ko -71.3332
R_ACALD_glc_ko -39.9912
R_PTAr_glc_ko -34.5215
R_CO2t_glc_ko -19.0668
R_GLUDy_glc_ko -4.53234
R_GLUt2r_glc_ko -4.53234
R_SUCOAS_glc_ko -4.53234
R_ACONTa_glc_ko  4.53234
R_ACONTb_glc_ko  4.53234
R_AKGDH_glc_ko  4.53234
R_AKGt2r_glc_ko  4.53234
R_CS_glc_ko   4.53234
R_FUM_glc_ko  4.53234
R_ICDHyr_glc_ko  4.53234
R_MDH_glc_ko  4.53234
R_NH4t_glc_ko  4.53234
R_SUCDi_glc_ko  4.53234
R_ATPM_glc_ko  8.39
R_PDH_glc_ko  10.0021
R_PYRt2_glc_ko  10.0021
R_O2t_glc_ko  22.9382
R_ACKr_glc_ko  34.5215
R_ACt2r_glc_ko  34.5215
R_ATPS4r_glc_ko  38.3791
R_NADH16_glc_ko  41.3441
R_CYTBD_glc_ko  45.8764
R_ALCD2x_glc_ko  62.2685
R_ETOHt2r_glc_ko  62.2685


In [21]:
solution.show_values(pattern='_nh4_ko', sort=True)

R_ALCD2x_nh4_ko -62.2685
R_ETOHt2r_nh4_ko -62.2685
R_ACKr_nh4_ko -34.5215
R_ACt2r_nh4_ko -34.5215
R_PGK_nh4_ko -16.4135
R_PGM_nh4_ko -15.17
R_PYRt2_nh4_ko -10.0021
R_CO2t_nh4_ko -5.5613
R_AKGt2r_nh4_ko -4.53234
R_RPI_nh4_ko -1.9744
R_SUCOAS_nh4_ko -1.45779
R_GLNS_nh4_ko  0.212537
R_GLUDy_nh4_ko  0.212537
R_O2t_nh4_ko  0.728897
R_BIOMASS_Ecoli_core_w_GAM_nh4_ko  0.831196
R_TKT2_nh4_ko  0.928175
R_TALA_nh4_ko  1.22824
R_TKT1_nh4_ko  1.22824
R_AKGDH_nh4_ko  1.45779
R_CYTBD_nh4_ko  1.45779
R_FUM_nh4_ko  1.45779
R_MDH_nh4_ko  1.45779
R_SUCDi_nh4_ko  1.45779
R_RPE_nh4_ko  2.15641
R_ACONTa_nh4_ko  2.35457
R_ACONTb_nh4_ko  2.35457
R_CS_nh4_ko   2.35457
R_ICDHyr_nh4_ko  2.35457
R_PYK_nh4_ko  2.35668
R_PPC_nh4_ko  2.38187
R_PIt2r_nh4_ko  3.05772
R_G6PDH2r_nh4_ko  4.13081
R_GND_nh4_ko  4.13081
R_PGL_nh4_ko  4.13081
R_GLUt2r_nh4_ko  4.53234
R_PGI_nh4_ko  5.69879
R_FBA_nh4_ko  7.79627
R_PFK_nh4_ko  7.79627
R_TPI_nh4_ko  7.79627
R_ATPM_nh4_ko  8.39
R_GLCpts_nh4_ko  10
R_ATPS4r_nh4_ko  11.3632
R_ENO_

In [23]:
solution.show_values(pattern='_bt', sort=True)

Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

## Step 3: Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [24]:
from reframed import SteadyCom
solution = SteadyCom(community, constraints=M9)

AttributeError: 'gurobipy.Model' object has no attribute 'linear_constraints'

In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [ ]:
print(solution)

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [ ]:
solution.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [ ]:
from reframed import fluxes2escher
fluxes2escher(solution.internal['glc_ko'])

In [ ]:
fluxes2escher(solution.internal['nh4_ko'])

**Exercise:** Look more closely at the compounds that are exchanged between the two mutants and also at their relative abundance. Is this what you expected? Do you think there could be different solutions?

## Step 4: Explore alternative solutions

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

![variability](../files/steadycom_variability.png)

> Would you expect a synthetic community to grow at its maximum growth rate?

**ReFramed** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [25]:
from reframed import SteadyComVA
variability = SteadyComVA(community, obj_frac=0.9, constraints=M9)

print('Strain\tMin\tMax')
for strain, (lower, upper) in variability.items():
    print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

AttributeError: 'gurobipy.Model' object has no attribute 'linear_constraints'

As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

In [ ]:
# Feel free to play around with these examples.
# Type your own code here...